<a href="https://colab.research.google.com/github/RobertGodin/CodePython/blob/master/ExempleReglageFinDistilBertTweetLORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installation des bibliothèques
%pip install numpy
%pip install pandas
%pip install transformers
%pip install datasets
%pip install peft
%pip install evaluate
%pip install torch
%pip install huggingface_hub
%pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00


In [1]:
# Chargement des données de tweet
nom_dataset = "mteb/tweet_sentiment_extraction"
import pandas as pd
from datasets import load_dataset
dataset_tweet = load_dataset(nom_dataset)
print(dataset_tweet)
df_ent = pd.DataFrame(dataset_tweet["train"])
df_ent.head(3)

C:\Users\vango\anaconda3b\envs\pytorch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})


,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative


In [2]:
# Chargement du modèle et tokenizer distilbert
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
nom_modele = 'distilbert-base-uncased'
# Créer un modèle pour la classification à partir de distilbert
# une nouvelle couche est ajoutée en sortie avec le nombre de classe = num_labels
modele = AutoModelForSequenceClassification.from_pretrained(nom_modele, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(nom_modele)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Vectorisation des textes pour l'entrainement
def tokenize_text(data):
  return tokenizer(data["text"], padding = "max_length",truncation=True)
dataset_vectorise = dataset_tweet.map(tokenize_text, batched=True)
print("Dataset vectorisé:",dataset_vectorise)
dataset_ent = dataset_vectorise["train"].shuffle(seed=42).select(range(500))  #petite collection
dataset_test = dataset_vectorise["test"].shuffle(seed=42).select(range(500))
df_ent=pd.DataFrame(dataset_ent)
df_ent.head(3)

Map: 100%|██████████| 3534/3534 [00:01<00:00, 3066.37 examples/s]


Dataset vectorisé: DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})


,id,text,label,label_text,input_ids,attention_mask
0,fd8e3ede29,what happen to the maids of yours? Quit once ...,1,neutral,"[101, 2054, 4148, 2000, 1996, 29229, 1997, 673...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,9206b5221d,Listening to the commentary track on Holiday I...,2,positive,"[101, 5962, 2000, 1996, 8570, 2650, 2006, 6209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,d6b51b539f,"chatting with some of my old classmates, helly...",1,neutral,"[101, 22331, 2007, 2070, 1997, 2026, 2214, 198...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [4]:
# Quelques exemples de prédictions avant entrainement
print("Predictions avant entrainement:")
print("--------------------------------------------------")
modele.to("cpu")
import torch
def predictions(liste_exemples):
  classe_etiquettes = {0: "negative", 1: "neutral", 2: "positive"}
  for texte in list_exemples :
    entrees = tokenizer.encode(texte, return_tensors="pt")
    logits = modele(entrees).logits
    predictions = torch.argmax(logits)
    sentiment = classe_etiquettes[predictions.item()]
    print("Analyse de sentiment pour '",texte,"' est :", sentiment)
liste_exemples = ["This is a very good comment.", "I don't recommend this restaurant.", 
                   "This is better that most !", "Don't ever go there !", "This is a good and bad thing."]
predictions(liste_exemples)

Analyse de sentiment pour 'This is a very nice tweet' est:  negative


In [5]:
import peft
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])
peft_config
modele = get_peft_model(modele, peft_config)
modele.print_trainable_parameters()

trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318


In [6]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
arguments_training = TrainingArguments(
    output_dir='dossier_bert_ft_tweet', # Dossier pour les résultats
    evaluation_strategy="epoch",
    num_train_epochs=10, # Défaut est 3
    learning_rate=2e-5, # Taux d'apprentissage
    per_device_train_batch_size=4, # Taille des mini-lots
    per_device_eval_batch_size=4,
    weight_decay=0.01, # Pour régularisation L2
    load_best_model_at_end=True
    
)
import evaluate
import numpy as np
exactitude = evaluate.load("accuracy")
def calcul_metriques(p):
    logits, etiquettes = p
    predictions = np.argmax(logits, axis=1)
    return exactitude.compute(predictions=predictions, references=etiquettes)

data_collator = DataCollatorWithPadding(tokenizer)
entraineur = Trainer(
    model=modele,
    args=arguments_training,
    train_dataset=dataset_ent,
    eval_dataset=dataset_test,
    data_collator=data_collator, # pour lots de taille fixe dynamiques
    compute_metrics=calcul_metriques
)
entraineur.train()

C:\Users\vango\anaconda3b\envs\pytorch_env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.081522,0.478000
2,No log,1.072672,0.460000
3,No log,1.059210,0.474000
4,1.057200,1.042728,0.472000
5,1.057200,1.026591,0.482000
6,1.057200,1.013061,0.490000
7,1.057200,0.996630,0.514000
8,0.963200,0.988198,0.522000
9,0.963200,0.980735,0.528000
10,0.963200,0.978009,0.526000


TrainOutput(global_step=1250, training_loss=0.991989794921875, metrics={'train_runtime': 5841.9406, 'train_samples_per_second': 0.856, 'train_steps_per_second': 0.214, 'total_flos': 672021964800000.0, 'train_loss': 0.991989794921875, 'epoch': 10.0})

In [8]:
# Les prédictions après entrainement pour les exemples
print("Predictions après entrainement:")
print("--------------------------------------------------")
modele.to("cpu")
predictions(liste_exemples)

Analyse de sentiment pour 'This is a very nice tweet' est:  neutral


In [10]:
modele

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [11]:
print("Analyse de sentiment pour 'This is a very nice tweet' est: ", analyse_sentiment('This is a very nice tweet'))

Analyse de sentiment pour 'This is a very nice tweet' est:  neutral
